In [1]:
# 2V25saC0pDFc8heT7wxsjigW3CF_32oQPdVZZP9wSMGd29FCa
# 2NqtkyBMF0KF99ofEU1fGz1pCJS_3XehxpoU47JGBiqY6tV7M

!pip install TTS openai-whisper pyngrok fastapi[all] nest_asyncio -q
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok
!ngrok authtoken 2NqtkyBMF0KF99ofEU1fGz1pCJS_3XehxpoU47JGBiqY6tV7M
!sudo apt update && sudo apt upgrade && sudo apt install ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.2/811.2 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.9/792.9 kB 17.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 18.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.3/36.3 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 8.8 MB/s eta 0:00:00
  Preparing metadat

In [2]:
import whisper
from fastapi import FastAPI, UploadFile, File, HTTPException, Body
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import nest_asyncio
from pyngrok import ngrok
import uuid
import os
import uvicorn
import torch
from TTS.api import TTS
from fastapi.responses import FileResponse
import asyncio
import gc

device = 'cuda' if torch.cuda.is_available() else 'cpu'

voice_model = whisper.load_model("large-v2").to(device)
tts = TTS('tts_models/en/jenny/jenny').to(device)
port=8888
gc.collect()

100%|██████████████████████████████████████| 2.87G/2.87G [00:29<00:00, 105MiB/s]


 > Downloading model to /root/.local/share/tts/tts_models--en--jenny--jenny


100%|██████████| 1.61G/1.61G [02:39<00:00, 10.1MiB/s]


 > Model's license - custom - see https://github.com/dioco-group/jenny-tts-dataset#important
 > Check https://opensource.org/licenses for more info.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:48000
 | > resample:False
 | > num_mels:100
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:2048
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:512
 | > win_length:2048


18

In [ ]:
app = FastAPI(title="SIH 2023 Backend")

app.add_middleware(
CORSMiddleware,
allow_origins=["*"],
allow_credentials=True,
allow_methods=["*"],
allow_headers=["*"],
)

class TTSModel(BaseModel):
    text: str
    emotion: str = "Cheerful & Professional"

@app.get("/")
async def home():
    return "SIH - LICHT DEN CODE"

@app.post("/transcribe/")
async def transcribe_audio(file: UploadFile = File(...)):
    audio_path = f"{uuid.uuid4()}.webm"
    with open(audio_path, "wb") as f:
        f.write(await file.read())
    result = voice_model.transcribe(whisper.pad_or_trim(whisper.load_audio(audio_path)))["text"]
    os.remove(audio_path)
    return {"text": result}

@app.post("/coqui-tts/")
async def coqui_tts(request: TTSModel = Body(...)):
    out = f"{uuid.uuid4()}.ogg"
    async def remove():
        loop = asyncio.get_event_loop()
        await loop.run_in_executor(None, lambda: os.remove(out))
    tts.tts_to_file(request.text, file_path=out, emotion=request.emotion)
    return FileResponse(out,headers={"Content-Disposition":f"attachment; filename={out}"},background=remove)

if __name__=="__main__":
    ngrok_tunnel = ngrok.connect(port)
    print('Public URL:', ngrok_tunnel.public_url)
    nest_asyncio.apply()
    uvicorn.run(app,port=port)

Public URL: https://8529-34-70-136-210.ngrok-free.app


INFO:     Started server process [299]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8888 (Press CTRL+C to quit)


INFO:     49.36.112.14:0 - "GET / HTTP/1.1" 200 OK
INFO:     2401:4900:1c21:d6ae:50d7:1519:7ea3:1919:0 - "POST /transcribe/ HTTP/1.1" 200 OK
INFO:     2401:4900:1c21:d6ae:50d7:1519:7ea3:1919:0 - "OPTIONS /coqui-tts/ HTTP/1.1" 200 OK
 > Text splitted to sentences.
['Pedro no quiere vivir conmigo.', 'Well, how are you?', 'I am fine.']
 > Processing time: 2.7073326110839844
 > Real-time factor: 0.4660582907701815
INFO:     2401:4900:1c21:d6ae:50d7:1519:7ea3:1919:0 - "POST /coqui-tts/ HTTP/1.1" 200 OK
INFO:     2401:4900:1c21:d6ae:50d7:1519:7ea3:1919:0 - "POST /transcribe/ HTTP/1.1" 200 OK
 > Text splitted to sentences.
['Pedro no quiere, well how are you?', 'Conmigo está para mí.']
 > Processing time: 1.057572364807129
 > Real-time factor: 0.19249588001585893
INFO:     2401:4900:1c21:d6ae:50d7:1519:7ea3:1919:0 - "POST /coqui-tts/ HTTP/1.1" 200 OK
INFO:     2401:4900:1c21:d6ae:50d7:1519:7ea3:1919:0 - "POST /transcribe/ HTTP/1.1" 200 OK
 > Text splitted to sentences.
['तुम कैसे हो?', 'I wan